# Repsly trial data

In [1]:
from repsly_data import RepslyData

repsly_data = RepslyData()
print('Reading data (this might take a minute or so)...', end='')
repsly_data.read_data_for_plain_nn('data/trial_users_analysis.csv')
print('done.')

Reading data (this might take a minute or so)...done.


Let's see what the data looks like:

In [2]:
read_batch = repsly_data.read_batch_for_plain_nn(batch_size=16)

X, y = next(read_batch)
print('X{}: {}'.format(list(X.shape), X))
print('X[0]: {}'.format(X[0]))

print('y:', y)


X[16, 241]: [[ 153.    1.    1. ...,    0.    0.    0.]
 [ 224.    0.    0. ...,    0.    0.    0.]
 [  54.    0.    0. ...,    0.    0.    0.]
 ..., 
 [ 448.    0.    0. ...,    0.    0.    0.]
 [  52.    0.    0. ...,    0.    0.    0.]
 [ 194.    0.    0. ...,    0.    0.    0.]]
X[0]: [ 153.    1.    1.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    7.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0

As you can see above, each input vector `X` has `1+15*16=241` values, most of which are zeros. The first one is the trial start date as offset from 2016-01-01 and the rest is different usage parameters for the following 16 days. Data provided by batch read is randomly shuffled. Output values are stored in `y` and they represent if the user purchased the Repsly service after the trial or not.